![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


In [48]:
WITH top_industries AS(
SELECT industry, count(i.company_id) AS num_unicorns
FROM public.industries AS i
INNER JOIN public.dates AS d
ON i.company_id = d.company_id
WHERE EXTRACT(Year FROM date_joined) IN (2019,2020,2021)
GROUP BY industry
ORDER BY num_unicorns DESC
LIMIT 3)

, valuation AS(SELECT industry, count(i.company_id) AS num_unicorns, EXTRACT(Year FROM date_joined) AS year,
AVG(valuation)AS average_valuation
FROM public.industries AS i
INNER JOIN public.dates AS d
ON i.company_id = d.company_id
INNER JOIN public.funding AS f
ON i.company_id = f.company_id		   
GROUP BY industry, year)

SELECT industry, year, num_unicorns, 
ROUND(average_valuation / 1000000000, 2) AS average_valuation_billions
FROM valuation
WHERE year IN ('2019','2020','2021')
AND industry IN (SELECT industry 
				FROM top_industries)
GROUP BY industry, year, num_unicorns, average_valuation_billions
ORDER BY industry, year DESC;

F